## Code to work with ATM data along the flowline of Zachariae Isstrom

**by Jukes Liu**

**June 2019 ICESat-2 Hackweek**


In [227]:
#IMPORT PACKAGES
import os
import glob
import pandas as pd
import csv
import numpy as np
import math
import matplotlib.pyplot as plt
import pyproj
import geopandas as gpd
from shapely.geometry import Point, Polygon
from ipyleaflet import Map, Marker, basemaps #'OpenTopoMap'

## Read in ATM elevation CSV files into dataframes and each variable into 1D arrays:

In [228]:
#get all the relevant csv files as a list and sort in order of the flight path
#filelist = glob.glob("../data/ILATM2_2018*.csv")
filelist = glob.glob("../data/ILATM2_2014*.csv")
filelist.sort(reverse=True)
#test with first file in the filelist
# filelist = [filelist[0]] 

#create an empty DataFrame to hold the stitched together ATM data along the profile
df_total = pd.DataFrame()

#for each csv file
for file in filelist:
    #print(file)
    #read the csv file into a pandas DataFrame
    csv_df = pd.read_csv(file, skiprows=9)
    
    #concatenate (join) it to the df_total
    df_total = pd.concat([csv_df, df_total])

#check that we have all the data properly stitched together:
#print(df_total)

#Read variables from csv DataFrame into 1D arrays
ATM_lat = df_total.loc[:, df_total.keys()[1]].values
ATM_long = df_total.loc[:, df_total.keys()[2]].values
ATM_elev = df_total.loc[:, df_total.keys()[3]].values
TrackID = df_total.loc[:, df_total.keys()[-1]].values

#Set in new DataFrame called ATM_data
ATM_data = pd.DataFrame([ATM_lat, ATM_long, ATM_elev, TrackID])
headers = ['lat', "lon", "elev", "TrackID"]
ATM_data = ATM_data.transpose()
ATM_data.columns = headers
#ATM_data.head()

#Select only the data with TrackID 0:
ATM_track0 = ATM_data[ATM_data['TrackID'] == 0.0].copy()

#get the variables in arrays:
track0_lat = ATM_track0.lat.values
track0_long = ATM_track0.lon.values
track0_elev = ATM_track0.elev.values

ATM_track0.head()

,lat,lon,elev,TrackID
3,79.202572,341.714609,32.5185,0.0
7,79.202270,341.714622,33.1235,0.0
11,79.201968,341.714638,32.7155,0.0
15,79.201666,341.714654,32.1009,0.0
19,79.201364,341.714670,30.7951,0.0


## Reproject lat and long into Greenland Polar Stereo coordinates (ESPG: 3413)

In [229]:
#Coordinate transformation function written by Fernando Paolo:
def transform_coord(proj1, proj2, x, y):
    """
    Transform coordinates from proj1 to proj2 (EPSG num).

    Example EPSG projs:
        Geodetic (lon/lat): 4326
        Polar Stereo AnIS (x/y): 3031
        Polar Stereo GrIS (x/y): 3413
    """
    # Set full EPSG projection strings
    proj1 = pyproj.Proj("+init=EPSG:"+str(proj1))
    proj2 = pyproj.Proj("+init=EPSG:"+str(proj2))
    return pyproj.transform(proj1, proj2, x, y)  # convert

#use the function to transform into Greenland Polar Stereo (PS_x, PS_y)
PS_x, PS_y = transform_coord(4326, 3413, track0_long, track0_lat)
print(PS_x)
print(PS_y)

[527314.0468452  527329.11751696 527344.24307394 ... 416008.10264363
 415976.20023373 415944.31738589]
[-1047781.64325332 -1047810.99688281 -1047840.32290794 ...
 -1102874.44909802 -1102873.40305985 -1102872.34885821]


## Calculate distance along the flowline:

In [230]:
#create an empty array to contain the distance along flow 
dist_along = np.empty(len(track0_lat), dtype=float)

#for each lat-long point:
for i in range(len(track0_lat)):
    #for the first point, set distance along flow to 0 m
    if i == 0:
        dist_along[i] = 0.0
    
    #for all other subsequent points:
    else:
        #calculate distance from the last point in meters
        x_diff = PS_x[i] - PS_x[i-1]
        y_diff = PS_y[i] - PS_y[i-1]
                
        #calculate distance from the previous point using the distance formula:    
        dist_last = math.sqrt(((x_diff)**2)+((y_diff)**2))
        #print(dist_last)
        
        #add it to the previous distance along the track to get the cumulative distance
        dist_along[i] = dist_last + dist_along[i-1]

    #check the array of distance along the track in meters:
    #print(dist_along[i], ATM_elev[i])
    
# PS_diffs = np.sqrt( np.diff(ref_x)**2 + np.diff(ref_y)**2 ) # m
# dist_along = np.cumsum(PS_diffs)
# dist_along = np.append(0, dist_along)

## Final data extracted into arrays
+ ATM_long: longitude of elev points
+ ATM_lat: latitude of elev points
+ PS_x: x-coords of elev points in Greenland Polar Stereo
+ PS_y: y-coords of elev points in Greenland Polar Stereo
+ ATM_elev: elevation points with z in meters
+ dist_along: distance along the flowline in meters

## ...and recombined into a dataframe called final_data:

In [231]:
variables = [track0_lat, track0_long, PS_x, PS_y, track0_elev, dist_along]
indices = ['ATM_lat', "ATM_long", "PS_x", "PS_y", "ATM_elev", "dist_along"]

#create DataFrame from these variables
final_data = pd.DataFrame(variables)
final_data = final_data.transpose()
final_data.columns = indices

#SAVE TO CSV
final_data.to_csv('/home/jovyan/xtrak/data/ATMprof_20140429.csv',index=False)
final_data.head()

,ATM_lat,ATM_long,PS_x,PS_y,ATM_elev,dist_along
0,79.202572,341.714609,527314.046845,-1.047782e+06,32.5185,0.000000
1,79.202270,341.714622,527329.117517,-1.047811e+06,33.1235,32.996374
2,79.201968,341.714638,527344.243074,-1.047840e+06,32.7155,65.993317
3,79.201666,341.714654,527359.368654,-1.047870e+06,32.1009,98.990275
4,79.201364,341.714670,527374.494258,-1.047899e+06,30.7951,131.987250


## Convert to GeoDataFrame

In [232]:
#create new geometry column:
final_data['geometry'] = list(zip(final_data['PS_x'], final_data['PS_y']))

#create new shapely Point geometry objects 
final_data['geometry'] = final_data['geometry'].apply(Point)
#final_data.head()

#convert DataFrame to GeoDataFrame in Polar Stereo
final_gdf = gpd.GeoDataFrame(final_data, crs={'init' :'epsg:3413'})

final_gdf.head()

,ATM_lat,ATM_long,PS_x,PS_y,ATM_elev,dist_along,geometry
0,79.202572,341.714609,527314.046845,-1.047782e+06,32.5185,0.000000,POINT (527314.0468452007 -1047781.643253321)
1,79.202270,341.714622,527329.117517,-1.047811e+06,33.1235,32.996374,POINT (527329.1175169554 -1047810.996882809)
2,79.201968,341.714638,527344.243074,-1.047840e+06,32.7155,65.993317,POINT (527344.2430739407 -1047840.322907937)
3,79.201666,341.714654,527359.368654,-1.047870e+06,32.1009,98.990275,POINT (527359.368654446 -1047869.648938797)
4,79.201364,341.714670,527374.494258,-1.047899e+06,30.7951,131.987250,POINT (527374.4942584686 -1047898.974975394)


## Plotting the data:

In [233]:
#2D elevation cross-section plot with DataFrame:
# elev_prof = final_data.plot(x='dist_along', y='ATM_elev', kind='scatter', s=0.05) 
# elev_prof.set_xlabel("Distance along glacier flowline (m)")
# elev_prof.set_ylabel("WGS84 Ellipsoid Height (m)")

#2D plot of track (X, Y, by elevation) with DataFrame:
track = final_data.plot(x='PS_x', y='PS_y', kind='scatter', s=0.05, c='ATM_elev', cmap='inferno')
track.set_xlabel("Polar Stereographic X (m)")
track.set_ylabel("Polar Stereographic Y (m)")

#plot with GeoDataFrame
# gdf_plot = final_gdf.plot('ATM_elev', s=0.05, cmap='inferno', legend=True);
# gdf_plot.set_xlabel("Polar Stereographic X (m)")
# gdf_plot.set_ylabel("Polar Stereographic Y (m)")

/srv/conda/lib/python3.6/site-packages/pandas/plotting/_core.py:304: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = self.plt.figure(figsize=self.figsize)


FigureCanvasNbAgg()

Text(0, 0.5, 'Polar Stereographic Y (m)')